In [ ]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('./titanic/train.csv')
len(df_train)

In [ ]:
df_train.info()

In [ ]:
df_train = df_train.drop(['Name', 'Ticket', 'PassengerId', 'Cabin'], axis=1)
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().any()

In [6]:
train_age_mean = float(round(df_train['Age'].mean()))
train_embarked_mean = 'S'
train_fare_mean = df_train['Fare'].mean()

In [7]:
df_train['Age'].fillna(train_age_mean, inplace=True)
df_train['Embarked'].fillna(train_embarked_mean, inplace=True) # as most frequent 'Embarked' 

In [8]:
df_train = pd.get_dummies(df_train, columns=['Sex', 'Embarked'])

In [9]:
features2standartization = ['Age', 'Fare']

tmp = df_train[features2standartization]
tmp_training_mean = tmp.mean()
tmp_training_std = tmp.std()

In [10]:
df_train[features2standartization] = (df_train[features2standartization] - tmp_training_mean) / tmp_training_std
df_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,-0.596720,1,0,-0.502163,0,1,0,0,1
1,1,1,0.633806,1,0,0.786404,1,0,1,0,0
2,1,3,-0.289088,0,0,-0.488580,1,0,0,0,1
3,1,1,0.403083,1,0,0.420494,1,0,0,0,1
4,0,3,0.403083,0,0,-0.486064,0,1,0,0,1


In [11]:
df_target = df_train['Survived']
df_features = df_train.drop('Survived', axis=1)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(df_features, df_target, test_size=0.2)
len(X_train), len(X_val)

(712, 179)

In [13]:
import torch

def get_acc(pred, gt):
    
    if isinstance(pred, torch.Tensor) or isinstance(gt, torch.Tensor):
        pred = pred.detach().numpy()
        gt = gt.detach().numpy()
    
    pred_class = (pred > 0.5).astype(int)   

    correct = (gt == pred_class).sum().item()
    return correct / len(gt)

In [14]:
from catboost import CatBoostRegressor

catboost = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    subsample=0.8,
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

catboost.fit(X_train, Y_train)

Y_prediction = catboost.predict(X_val)
get_acc(Y_prediction, Y_val)

0:	learn: 0.4798512	total: 142ms	remaining: 2m 21s
100:	learn: 0.3465620	total: 223ms	remaining: 1.99s
200:	learn: 0.3288531	total: 303ms	remaining: 1.21s
300:	learn: 0.3136341	total: 388ms	remaining: 900ms
400:	learn: 0.2994068	total: 472ms	remaining: 706ms
500:	learn: 0.2869891	total: 557ms	remaining: 555ms
600:	learn: 0.2762736	total: 643ms	remaining: 427ms
700:	learn: 0.2674239	total: 728ms	remaining: 310ms
800:	learn: 0.2583468	total: 829ms	remaining: 206ms
900:	learn: 0.2510396	total: 919ms	remaining: 101ms
999:	learn: 0.2440096	total: 1s	remaining: 0us


0.8770949720670391

In [15]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_val)
get_acc(Y_prediction, Y_val)

0.8100558659217877

### MLP

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

X_tr_tensor = torch.Tensor(X_train.values)
Y_tr_tensor = torch.Tensor(Y_train.values)

X_vl_tensor = torch.Tensor(X_val.values)
Y_vl_tensor = torch.Tensor(Y_val.values)

X_tr_tensor.shape, Y_tr_tensor.shape, X_vl_tensor.shape, Y_vl_tensor.shape

(torch.Size([712, 10]),
 torch.Size([712]),
 torch.Size([179, 10]),
 torch.Size([179]))

In [17]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_dataset = TensorDataset(X_tr_tensor, Y_tr_tensor)
val_dataset = TensorDataset(X_vl_tensor, Y_vl_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class BCModel(nn.Module):
    def __init__(self, input_dim):
        super(BCModel, self).__init__()
        
        mid_shape = 32
        self.fc1 = nn.Linear(input_dim, mid_shape)
        self.fc2 = nn.Linear(mid_shape, 1)
        
    def forward(self, x):
        out = self.fc1(x)
        out = torch.tanh(out)
        out = self.fc2(out).squeeze(1)
        return out
    
model = BCModel(10)
model

In [ ]:
sum([p.nelement() for p in model.parameters()])

In [ ]:
import torch.optim as optim

_lr = 3e-2

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(params=model.parameters(), lr=_lr)

In [ ]:
test_x = X_tr_tensor[:batch_size]
test_y = Y_tr_tensor[:batch_size]

out = model(test_x)

print(f'Input shape: {test_x.shape}; label shape: {test_y.shape}; out shape: {out.shape} \n')

loss = criterion(out, test_y)
acc = get_acc(out, test_y)

print(f'Loss: {loss.item():.4f}, acc: {acc:.4f}')

In [ ]:
current_eph = 0
t_loss_eph = []
t_acc_eph = []

v_loss_eph = []
v_acc_eph = []

In [ ]:
for eph in range(current_eph, current_eph + 30):
    print(f" {'-' * 5} Epoch: {eph} {'-' * 5}")
    current_eph += 1
    t_losses = []
    t_acc = []

    for batch, label in train_dataloader:
        out = model(batch)
        prob = torch.sigmoid(out)
        
        loss = criterion(out, label)
        t_losses.append(loss.item())
        t_acc.append(get_acc(prob, label))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss_eph = sum(t_losses) / len(t_losses)
    acc_eph = sum(t_acc) / len(t_acc)
    
    t_loss_eph.append(loss_eph)
    t_acc_eph.append(acc_eph)
    
    print(f'Train loss: {loss_eph:.4f}; acc: {acc_eph:.4f}')
    
    v_losses = []
    v_acc = []
    for batch, label in val_dataloader:
        out = model(batch)
        prob = torch.sigmoid(out)
        
        loss = criterion(out, label)
        v_losses.append(loss.item())
        v_acc.append(get_acc(prob, label))
        
    loss_eph = sum(v_losses) / len(v_losses)
    acc_eph = sum(v_acc) / len(v_acc)
    
    v_loss_eph.append(loss_eph)
    v_acc_eph.append(acc_eph)    
    
    print(f'Val loss: {loss_eph:.4f}; acc: {acc_eph:.4f}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(t_loss_eph, label='train')
plt.plot(v_loss_eph, label='val')
plt.legend()
plt.show()

In [ ]:
plt.plot(t_acc_eph, label='train')
plt.plot(v_acc_eph, label='val')
plt.legend()
plt.show()

### MLP from scratch

In [64]:
input_dim = X.shape[1]
mid_dim = 32

# W1 = np.random.uniform(0, 1, (input_dim, mid_dim)).astype(np.float32) * 0.01
# b1 = np.random.uniform(0, 1, (mid_dim)).astype(np.float32) 

# W2 = np.random.uniform(0, 1, (mid_dim, 1)).astype(np.float32) * 0.01
# b2 = np.random.uniform(0, 1, (1)).astype(np.float32)

# parameters = [W1, b1, W2, b2]
# sum([p.size for p in parameters])

# W1.shape, b1.shape, W2.shape, b2.shape

In [65]:
W1 = torch.rand((input_dim, mid_dim)) * 0.01
b1 = torch.rand((mid_dim)) 

W2 = torch.rand((mid_dim, 1)) * 0.01
b2 = torch.rand((1))

W1.shape, b1.shape, W2.shape, b2.shape

(torch.Size([10, 32]), torch.Size([32]), torch.Size([32, 1]), torch.Size([1]))

In [66]:
parameters = [W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

sum([p.nelement() for p in parameters])

385

In [67]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [68]:
_batch_size = 32
_lr = 5e-2

current_eph = 0
t_loss_eph = []
t_acc_eph = []

v_loss_eph = []
v_acc_eph = []

In [69]:
for eph in range(current_eph, current_eph + 30):
    print(f" {'-' * 5} Epoch: {eph} {'-' * 5}")
    current_eph += 1
    t_losses = []
    t_acc = []

    for i in range(int(len(X_train) / _batch_size) + 1):
        batch_idx = np.arange(_batch_size * i, min(_batch_size * (i + 1), len(X_train)))
        
        X = X_tr_tensor[batch_idx]
        Y = Y_tr_tensor[batch_idx].unsqueeze(1)
        #Y = np.expand_dims(Y_train.values[batch_idx], 1) # convert shape (32) to (32, 1)
        
        Z1 = X @ W1 + b1
        A1 = (math.e**Z1 - math.e**(-Z1)) / (math.e**Z1 + math.e**(-Z1)) # tanh
        Z2 = A1 @ W2 + b2

        probs = 1 / (1 + math.e**(-Z2))
        
        loss = - (Y * probs.log() + (1 - Y) * (1 - probs).log()).sum() / len(Y)
        t_losses.append(loss)
        t_acc.append(get_acc(probs, Y))
        
        dZ2 = (probs - Y) / X.size(0)
        dW2 = A1.T @ dZ2
        db2 = dZ2.sum(axis=0)

        dA1 = dZ2 @ W2.T
        dZ1 = dA1 * (1 - A1**2)

        dW1 = X.T @ dZ1
        db1 = dZ1.sum(axis=0)
        
        grads = [dW1, db1, dW2, db2]

        # for p in parameters:
        #     p.grad = None
        # loss.backward()
        # cmp('w2', dW2, W2)
        # cmp('b2', db2, b2)
        # cmp('w1', dW1, W1)
        # cmp('b1', db1, b1)
        
        for param, grad in zip(parameters, grads):
            param.data += -_lr * grad

    loss_eph = sum(t_losses) / len(t_losses)
    acc_eph = sum(t_acc) / len(t_acc)
    
    t_loss_eph.append(loss_eph)
    t_acc_eph.append(acc_eph)
    
    print(f'Train loss: {loss_eph:.4f}; acc: {acc_eph:.4f}')
    
    v_losses = []
    v_acc = []
    for i in range(int(len(X_train) / _batch_size) + 1):
        batch_idx = np.arange(_batch_size * i, min(_batch_size * (i + 1), len(X_train)))
        
        X = X_tr_tensor[batch_idx]
        Y = Y_tr_tensor[batch_idx].unsqueeze(1)
        
        Z1 = X @ W1 + b1
        A1 = (math.e**Z1 - math.e**(-Z1)) / (math.e**Z1 + math.e**(-Z1)) # tanh
        Z2 = A1 @ W2 + b2

        probs = 1 / (1 + math.e**(-Z2))
        
        loss = - (Y * probs.log() + (1 - Y) * (1 - probs).log()).sum() / len(Y)
        
        v_losses.append(loss)
        v_acc.append(get_acc(probs, Y))
        
    loss_eph = sum(v_losses) / len(v_losses)
    acc_eph = sum(v_acc) / len(v_acc)
    
    v_loss_eph.append(loss_eph)
    v_acc_eph.append(acc_eph)    
    
    print(f'Val loss: {loss_eph:.4f}; acc: {acc_eph:.4f}')

 ----- Epoch: 0 -----
w2              | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
b2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
w1              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
b1              | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-10
w2              | exact: False | approximate: True  | maxdiff: 2.2351741790771484e-08
b2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
w1              | exact: False | approximate: True  | maxdiff: 6.984919309616089e-10
b1              | exact: False | approximate: True  | maxdiff: 1.7462298274040222e-10
w2              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
b2              | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
w1              | exact: False | approximate: True  | maxdiff: 3.4924596548080444e-10
b1              | exact: False | ap

w2              | exact: False | approximate: True  | maxdiff: 8.614733815193176e-09
b2              | exact: True  | approximate: True  | maxdiff: 0.0
w1              | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
b1              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
Train loss: 0.6635; acc: 0.6196
Val loss: 0.6613; acc: 0.6196
 ----- Epoch: 3 -----
w2              | exact: False | approximate: False | maxdiff: 1.1874362826347351e-08
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
b1              | exact: False | approximate: True  | maxdiff: 8.731149137020111e-10
w2              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
b2              | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
w1              | exact: False | approximate: True  | maxdiff: 2.561137080192566e

w2              | exact: False | approximate: True  | maxdiff: 1.30385160446167e-08
b2              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
w1              | exact: False | approximate: True  | maxdiff: 1.8044374883174896e-09
b1              | exact: False | approximate: True  | maxdiff: 6.984919309616089e-10
w2              | exact: False | approximate: True  | maxdiff: 2.2351741790771484e-08
b2              | exact: True  | approximate: True  | maxdiff: 0.0
w1              | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
b1              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
w2              | exact: False | approximate: True  | maxdiff: 2.2351741790771484e-08
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
b1              | exact: False | approximate: True  | maxdiff: 1.04773789644241

w2              | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
b2              | exact: True  | approximate: True  | maxdiff: 0.0
w1              | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
b1              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
w2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b2              | exact: False | approximate: True  | maxdiff: 2.2351741790771484e-08
w1              | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
b1              | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
w2              | exact: False | approximate: True  | maxdiff: 1.0710209608078003e-08
b2              | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
w1              | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
b1              | exact: False | approximate: True  | maxdiff: 1.16415321826

b1              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
w2              | exact: False | approximate: True  | maxdiff: 2.2351741790771484e-08
b2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
w1              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
b1              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
w2              | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b1              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
w2              | exact: False | approximate: True  | maxdiff: 1.6763806343078613e-08
b2              | exact: False | approximate: True  | maxdiff: 6.51925802230835e-09
w1              | exact: False | approximate: True  | maxdiff:

...

Val loss: 0.5550; acc: 0.7188
 ----- Epoch: 19 -----
w2              | exact: False | approximate: True  | maxdiff: 6.51925802230835e-09
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b1              | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
w2              | exact: False | approximate: True  | maxdiff: 9.778887033462524e-09
b2              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-08
w1              | exact: False | approximate: True  | maxdiff: 1.0244548320770264e-08
b1              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
w2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b2              | exact: True  | approximate: True  | maxdiff: 0.0
w1              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b1              | exact: Fa

w1              | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
b1              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w2              | exact: False | approximate: True  | maxdiff: 1.3969838619232178e-08
b2              | exact: False | approximate: True  | maxdiff: 1.6763806343078613e-08
w1              | exact: False | approximate: False | maxdiff: 1.862645149230957e-08
b1              | exact: False | approximate: True  | maxdiff: 4.6566128730773926e-09
w2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b2              | exact: True  | approximate: True  | maxdiff: 0.0
w1              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b1              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
w2              | exact: False | approximate: True  | maxdiff: 2.60770320892334e-08
b2              | exact: False | approximate: True  | maxdiff: 3.7252902984619

w2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 1.210719347000122e-08
b1              | exact: False | approximate: True  | maxdiff: 4.6566128730773926e-09
w2              | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b1              | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
w2              | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
w1              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b1              | exact: False | approximate: True  | maxdi

w2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
w1              | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
b1              | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
w2              | exact: False | approximate: True  | maxdiff: 4.6566128730773926e-09
b2              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-09
w1              | exact: False | approximate: False | maxdiff: 1.7229467630386353e-08
b1              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
w2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b2              | exact: True  | approximate: True  | maxdiff: 0.0
w1              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b1              | exact: False | approximate: True  | maxdiff: 3.7252902984619

w2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b2              | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
w1              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-08
b1              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
w2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
b2              | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
w1              | exact: False | approximate: True  | maxdiff: 2.60770320892334e-08
b1              | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
w2              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-09
b2              | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
w1              | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
b1              | exact: False | approximate: True  | maxdiff: 7.

### Inference

In [ ]:
df_test = pd.read_csv('./titanic/test_y.csv')
df_test = df_test.drop(['Name', 'Ticket', 'PassengerId', 'Cabin'], axis=1)
df_test.head()

In [ ]:
df_test.isnull().any()

In [ ]:
df_test['Age'].fillna(30, inplace=True)
df_test['Embarked'].fillna('S', inplace=True) # as most frequent 'Embarked' 
df_test['Fare'].fillna(train_fare_mean, inplace=True)

df_test[features2standartization] = (df_test[features2standartization] - tmp_training_mean) / tmp_training_std

In [ ]:
df_test = pd.get_dummies(df_test, ['Sex', 'Embarked'])

In [ ]:
df_test_y = df_test['Survived']
df_test_features = df_test.drop('Survived', axis=1)

In [ ]:
df_features.head()

In [ ]:
df_test_features.head()

In [ ]:
df_features.describe() - df_test_features.describe()

In [ ]:
X_test = df_test_features.values
Y_test = df_test_y.values

X_test.shape, Y_test.shape

In [ ]:
Y_pred = catboost.predict(df_test_features)
get_acc(Y_pred, df_test_y)

In [ ]:
Y_pred = random_forest.predict(df_test_features)
get_acc(Y_pred, df_test_y)

In [ ]:
X_test = torch.Tensor(df_test_features.values)
Y_test = torch.Tensor(df_test_y.values)

X_test.shape, Y_test.shape

In [ ]:
pred = model(X_test)
pred = torch.sigmoid(pred)
pred.shape

In [ ]:
get_acc(pred, Y_test)